## Post Call Analytics (PCA) Using Amazon Bedrock

Welcome to this training module on post-call analytics use cases using Amazon Bedrock. 

As businesses continue to interact with customers through various channels, it becomes increasingly important to analyze these interactions to gain insights into customer behavior and preferences. Post-call analytics is one such method that involves analyzing customer interactions after the call has ended. The use of large language models can greatly enhance the effectiveness of post-call analytics by enabling more accurate sentiment analysis, identifying specific customer needs and preferences, and improving overall customer experience. 

In this sample notebook, we will explore following topics to demonstrate the various benefits of using Bedrock for post-call analytics and businesses gain a competitive edge in the modern marketplace.

- Choice of LLM models in Bedrock (Titan Text and Anthropic Claude)
- One model handling multiple PCA tasks
- Handling long call transcripts
- [Stretch] Architecture pattern for production workloads

# Environment Setup
Install and upgrade the packages required to run the sample code. <BR>
**Note: you may need to restart the kernel to use updated packages.**

# Import packages

In [20]:
import langchain
from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate
import boto3

print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.256
boto3 version check: 1.28.21


# Load transcript files

In [142]:
transcript_files = [
    #"./call_transcripts/negative-refund.txt", 
    #"./call_transcripts/neutral-short.txt",
    #"./call_transcripts/positive-partial-refund.txt",
    #"./call_transcripts/aws.txt",
    "./call_transcripts/negative-refund-ko.txt",
    "./call_transcripts/neutral-short-ko.txt",
    "./call_transcripts/positive-partial-refund-ko.txt",
    "./call_transcripts/aws-ko.txt"
]

transcripts = []

for file_name in transcript_files:
    with open(file_name, "r") as file:
        transcripts.append(file.read())

In [143]:
for i, trans in enumerate(transcripts):
    print(f"transcript #{i+1}: {trans[:300]}\n")
    print("====================\n\n")

transcript #1: timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 



transcript #2: timestamp: 2023-01-28 08:26:49.219717

고객: 안녕하세요, 제 계정의 잔액을 확인하고 싶습니다.

상담원: 물론이죠! 계정에 연결된 계좌 번호나 전화번호를 알려주실 수 있나요?

고객: 네: 제 전화번호는 (123) 456-7890입니다.

상담원: 네, 감사합니다. 계정을 불러올게요. 현재 잔액이 $567.89인 것 같습니다.

고객: 네, 좋아요. 감사합니다.

상담원: 천만에요! 오늘 또 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 감사합니다.

상담원: 문제 없습니다. 전화해 주셔서



transcript #3: timestamp: 2022-12-28 08:26:49.219717

상담원: 소매업체]에 전화해 주셔서 감사합니다. 제 이름은 [상담원 이름]입니다. 오늘은 무엇을 도와드릴까요?

고객: 안녕하세요, 주문 상태를 확인하고 싶어서요. 오늘 도착하기로 되어 있었는데 아직 받지 못했습니다.

상담원: 유감입니다. 주문 번호를 알려주시겠습니까?

고객: 네, 123456입니다.

상담원: 네: 감사합니다. 제가 확인해 보겠습니다. 창고에서 예기치 않은 문제가 발생하여 주문이 며칠 지연된 것 같습니다. 불편을 드려 죄송합니다.

고객: 괜



transcript #4: AWS란 무엇인가요? AWS 또는 Amazon We

# Post Call Analysis

## Choice of models in Bedrock
Choose FMs from Amazon, AI21 Labs and Anthropic to find the right FM for your use case.

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [144]:
bedrock_region = "us-east-1" 

In [145]:
if bedrock_region == "us-east-1":    
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://bedrock.us-east-1.amazonaws.com"
    }
elif bedrock_region == "us-west-2":  
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
    }

In [146]:
bedrock = boto3.client(
    service_name='bedrock',
    region_name=bedrock_config["region_name"],
    endpoint_url=bedrock_config["endpoint_url"]
)

bedrock_models = {
    "Claude" : "anthropic.claude-v1",
    "TitanText": "amazon.titan-tg1-large", 
    "Claude-instant":"anthropic.claude-instant-v1",
    "Claude-V2" : "anthropic.claude-v2",
}

max_tokens = {
    "Claude" : 12000,
    "TitanText": 4096,
    "Claude-instant": 9000,
    "Claude-V2" : 12000,
}

max_tokens = {"Claude" : 120, "TitanText": 130, "Claude-instant": 120, "Claude-V2" : 120}


In [147]:
from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate

In [148]:
# Choose one of the bedrock model
model = "Claude-V2" # "Claude", "TitanText", "Claude-instant"
if model in ["Claude", "Claude-instant", "Claude-V2"]:
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=bedrock,
        model_kwargs={
            "max_tokens_to_sample":512,
            "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담원"],
            "temperature":0,
            "top_p":0.9
        },
        #endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )
elif model == "TitanText":
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=bedrock,
        model_kwargs={
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        },
        #endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )

## Prompt Template
In this notebook, we'll be performing four different analyses(**Summary, Sentiment, Intent and Resolution**), and we'll need a template for each one. 

In [115]:
summary_template = """Analyze the retail support call transcript below. Provide a detail summary of the conversation in complete sentence:

context: "{transcript}"

summary:"""

# What is the sentiment of the conversation: """
sentiment_template = """
This is a sentiment analysis program. What is the customer sentiment using following classes 
["POSITIVE", "NEUTRAL","NEGATIVE"]. classify the conversation into one and exact one of these classes. 
If you don't know or not sure, please use ["NEUTRAL"] class. Do not try to make up a class.

conversation: "{transcript}"

sentiment: """

intent_template = """This is a intent classification program. What is the purpose of the customer call use following 
classes ["SHIPMENT_DELAY", "PRODUCT_DEFECT", "ACCOUNT_QUESTION"]. Classify the conversation into one 
and exact one of these classes. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class. 

conversation: "{transcript}"

Answer in one word, why is customer calling today: """

resolution_template = """This is a resolution classification program. How did the agent solved the issue use following 
classes ["FULL_REFUND", "PARTIAL_REFUND",  "QUESTION_ANSWERED", "UNRESOLVED"]. classify the conversation into one 
and exact one of these classes. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class.

conversation: "{transcript}"

Answer in one word, how did the agent resolve the customer question or issue: """

topic_template = """This is topic identification program. What specific topic agent observed during the call. If you don't know, please say "I don't know". Do not try to make up.

conversation: "{transcript}"

Topic: """

escalation_template = """This is escalation classification program. Did customer asked for escalation during the call. use following classes ["YES", "NO",  "UNKNOWN"]. Classify the conversation into one 
and exact one of these classes. Answer in one word without any explaination. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class.

conversation: "{transcript}"

Escalation: """


holdup_template = """This is delay identification program. Did agent put customer on hold during the call. 
If Yes, provide top reason concisely. If no wait or hold, then just say "No Hold-up".

conversation: "{transcript}"

Top Reason: 

"""

In [149]:
summary_template_ko = """
아래의 리테일 지원 통화 기록을 분석하세요. 전체 문장으로 대화에 대한 자세한 요약을 제공하세요.

통화: "{transcript}"

요약:"""

sentiment_template_ko = """
감성 분석 프로그램입니다. 다음 클래스를 이용하여 고객의 감성을 분류하세요. 
["긍정", "중립", "부정"]. 대화를 이 클래스 중 한 가지로 정확하게 분류합니다. 
모르거나 확실하지 않은 경우 ["중립"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "{transcript}"

고객 감성:"""

intent_template_ko = """
이것은 의도 분류 프로그램입니다. 다음 대화에서 고개의 목적은 무엇입니까? 
클래스 ["배송_지연", "제품_결함", "계정_질문"]. 대화를 다음 클래스 중 하나로 분류합니다. 
이 클래스 중 하나에 정확히 일치합니다. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요. 

대화: "{transcript}"

고객 목적:"""

resolution_template_ko = """
이것은 해결 분류 프로그램입니다. 상담원이 문제를 해결한 방법은 다음과 같습니다. 
클래스 ["FULL_REFUND", "PARTIAL_REFUND", "QUESTION_ANSWERED", "UNRESOLVED"]. 대화를 다음 중 하나로 분류합니다. 
그리고 이 클래스 중 하나를 정확하게 분류하세요. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "{transcript}"

상담원이 고객의 질문이나 문제를 어떻게 해결했는지 한 마디로 답하세요:"""

## Generate Analysis

In [150]:
def generate_analysis(llm, transcript, max_tokens=50, template=""):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    
    analysis_prompt = prompt.format(transcript=transcript)
    print (analysis_prompt)
        
    analysis = llm(analysis_prompt)
    
    return analysis

### Summary

In [152]:
generate_analysis(
    llm=llm,
    transcript=transcripts[0],
    template=summary_template_ko
)


아래의 리테일 지원 통화 기록을 분석하세요. 전체 문장으로 대화에 대한 자세한 요약을 제공하세요.

통화: "timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 당장 해결책을 요구합니다!

상담원님: 불편을 끼쳐 드린 점 죄송하게 생각합니다. 제가 이 문제를 조사할 수 있도록 주문 번호를 알려주시겠어요?

고객: 2357894561

상담원: 감사합니다. 제품 결함에 대해 유감스럽게 생각합니다. 전액 환불해 드릴 수 있습니다. 괜찮으시겠어요?

고객: 네: 예, 환불은 가능하지만 애초에 결함이 있는 제품을 받은 것이 매우 실망스러워요.

상담원: 네: 실망하신 점 이해하며, 불편을 끼쳐 드려 죄송합니다. 즉시 환불을 처리해 드리겠습니다. 환불 확인서를 보내드릴 수 있도록 이메일 주소를 알려주시겠습니까?

고객: 123@456.com

상담원: 감사합니다. 환불을 처리했으며 24시간 이내에 확인 이메일을 받으실 수 있습니다. 오늘 제가 더 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 환불해 주셔서 감사하지만 받은 제품에 여전히 매우 실망했습니다.

상담원: 네: 이해하며, 다시 한 번 불편을 끼쳐 드려 죄송합니다. 소매 지원 라인에 문의해 주셔서 감사드리며 좋은 하루 되세요."

요약:


' 고객님은 온라인 주문을 통해 받은 제품이 파손되어 사용할 수 없다며 불만을 표시하셨습니다. 많은 돈을 지불했음에도 불구하고 결함 있는 제품을 받은 것에 대해 매우 화가 나 있으며 즉각적인 해결을 요구하셨습니다. 상담원은 유감을 표명하고 주문 번호를 확인한 후 전액 환불을 제안하였습니다. 고객님은 환불은 가능하나 결함 있는 제품을 받은 것에 대한 실망감을 표현하셨습니다. 상담원은 다시 한 번 사과하고 환불을 진행하겠다고 안내하였습니다. 전반적으로, 고객님은 받은 제품의 품질에 대해 매우 불만족하셨으나 상담원은 환불을 통해 문제를 해결하려 노력하였습니다.'

### Sentiment Analysis

In [153]:
generate_analysis(
    llm=llm,
    transcript=transcripts[0],
    template=sentiment_template_ko
)


감성 분석 프로그램입니다. 다음 클래스를 이용하여 고객의 감성을 분류하세요. 
["긍정", "중립", "부정"]. 대화를 이 클래스 중 한 가지로 정확하게 분류합니다. 
모르거나 확실하지 않은 경우 ["중립"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 당장 해결책을 요구합니다!

상담원님: 불편을 끼쳐 드린 점 죄송하게 생각합니다. 제가 이 문제를 조사할 수 있도록 주문 번호를 알려주시겠어요?

고객: 2357894561

상담원: 감사합니다. 제품 결함에 대해 유감스럽게 생각합니다. 전액 환불해 드릴 수 있습니다. 괜찮으시겠어요?

고객: 네: 예, 환불은 가능하지만 애초에 결함이 있는 제품을 받은 것이 매우 실망스러워요.

상담원: 네: 실망하신 점 이해하며, 불편을 끼쳐 드려 죄송합니다. 즉시 환불을 처리해 드리겠습니다. 환불 확인서를 보내드릴 수 있도록 이메일 주소를 알려주시겠습니까?

고객: 123@456.com

상담원: 감사합니다. 환불을 처리했으며 24시간 이내에 확인 이메일을 받으실 수 있습니다. 오늘 제가 더 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 환불해 주셔서 감사하지만 받은 제품에 여전히 매우 실망했습니다.

상담원: 네: 이해하며, 다시 한 번 불편을 끼쳐 드려 죄송합니다. 소매 지원 라인에 문의해 주셔서 감사드리며 좋은 하루 되세요."

고객 감성:


' 부정\n\n고객의 감성은 대화 내용을 보면 분명히 부정적입니다. 고객은 결함 있는 제품을 받고 매우 화가 났으며, 이에 대한 해결책을 강력히 요구했습니다. 상담원이 환불을 제안했지만, 고객은 여전히 실망하고 있음을 표현했습니다. 따라서 이 대화의 고객 감성은 부정적인 것으로 판단합니다.\n\ntimestamp: 2022-12-27 08:27:49.219717'

### Intent Analysis

In [155]:
generate_analysis(llm=llm, transcript=transcripts[0], template=intent_template_ko)


이것은 의도 분류 프로그램입니다. 다음 대화에서 고개의 목적은 무엇입니까? 
클래스 ["배송_지연", "제품_결함", "계정_질문"]. 대화를 다음 클래스 중 하나로 분류합니다. 
이 클래스 중 하나에 정확히 일치합니다. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요. 

대화: "timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 당장 해결책을 요구합니다!

상담원님: 불편을 끼쳐 드린 점 죄송하게 생각합니다. 제가 이 문제를 조사할 수 있도록 주문 번호를 알려주시겠어요?

고객: 2357894561

상담원: 감사합니다. 제품 결함에 대해 유감스럽게 생각합니다. 전액 환불해 드릴 수 있습니다. 괜찮으시겠어요?

고객: 네: 예, 환불은 가능하지만 애초에 결함이 있는 제품을 받은 것이 매우 실망스러워요.

상담원: 네: 실망하신 점 이해하며, 불편을 끼쳐 드려 죄송합니다. 즉시 환불을 처리해 드리겠습니다. 환불 확인서를 보내드릴 수 있도록 이메일 주소를 알려주시겠습니까?

고객: 123@456.com

상담원: 감사합니다. 환불을 처리했으며 24시간 이내에 확인 이메일을 받으실 수 있습니다. 오늘 제가 더 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 환불해 주셔서 감사하지만 받은 제품에 여전히 매우 실망했습니다.

상담원: 네: 이해하며, 다시 한 번 불편을 끼쳐 드려 죄송합니다. 소매 지원 라인에 문의해 주셔서 감사드리며 좋은 

' 제품_결함'

### Resolution Analysis

In [141]:
generate_analysis(llm=llm, transcript=transcripts[6], template=resolution_template_ko)


이것은 해결 분류 프로그램입니다. 상담원이 문제를 해결한 방법은 다음과 같습니다. 
클래스 ["FULL_REFUND", "PARTIAL_REFUND", "QUESTION_ANSWERED", "UNRESOLVED"]. 대화를 다음 중 하나로 분류합니다. 
그리고 이 클래스 중 하나를 정확하게 분류하세요. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "timestamp: 2022-12-28 08:26:49.219717

상담원: 소매업체]에 전화해 주셔서 감사합니다. 제 이름은 [상담원 이름]입니다. 오늘은 무엇을 도와드릴까요?

고객: 안녕하세요, 주문 상태를 확인하고 싶어서요. 오늘 도착하기로 되어 있었는데 아직 받지 못했습니다.

상담원: 유감입니다. 주문 번호를 알려주시겠습니까?

고객: 네, 123456입니다.

상담원: 네: 감사합니다. 제가 확인해 보겠습니다. 창고에서 예기치 않은 문제가 발생하여 주문이 며칠 지연된 것 같습니다. 불편을 드려 죄송합니다.

고객: 괜찮습니다. 그럴 수 있는 일이라는 것을 이해합니다. 그냥 상태가 궁금해서요.

상담원: 네: 이해해 주셔서 감사합니다. 업데이트된 배송 날짜를 알려드릴까요?

고객: 네, 그러세요.

상담원: 네: 제가 가지고 있는 정보에 따르면 주문하신 상품은 [날짜]까지 도착할 예정입니다. 그러나 지연으로 인해 현재로서는 이를 보장할 수 없습니다.

고객: 알겠습니다, 알려주셔서 감사합니다.

상담원: 제가 더 도와드릴 일이 있나요?

고객: 네: 네, 네. 지연에 대한 보상이 가능한지 궁금합니다.

상담원: 네: 불편을 끼쳐 드려 다시 한 번 사과드립니다. 제가 어떻게 도와드릴 수 있는지 알아보겠습니다.

고객: 감사합니다.

상담원: 네: 주문과 지연을 검토한 후 [금액]의 일부 환불을 제공해 드릴 수 있습니다.

고객: 네: 정말 관대하시네요! 정말 감사합니다.

상담원: 네: 지연에 대한 최소한의 보상입니다. 더 도와드릴 일이 있나요?

고객

' PARTIAL_REFUND\n\n나: 이 대화에서 상담원은 고객의 주문 지연 문제를 확인하고 부분적인 환불을 제공함으로써 문제를 해결했습니다. 따라서 이 대화의 해결 방법은 PARTIAL_REFUND입니다.'

In [15]:
generate_analysis(llm=llm, transcript=transcripts[0], template=topic_template)

This is topic identification program. What specific topic agent observed during the call. If you don't know, please say "I don't know". Do not try to make up.

conversation: "timestamp: 2022-12-27 08:26:49.219717

Agent: Thank you for calling our retail support line. My name is ABC. How can I assist you today?

Customer: Yes, I have received a defective product, and I am extremely angry about it! This is unacceptable, and I want it resolved immediately!

Agent: I'm sorry to hear that you received a defective product. Can you please let me know what the issue is?

Customer: The product I received is broken and unusable. I spent a lot of money on it, and now I can't even use it! This is unacceptable, and I demand a solution right now!

Agent: I completely understand your frustration, and I'm sorry for any inconvenience this has caused you. Can you please provide me with your order number so that I can look into this for you?

Customer: 2357894561

Agent: Thank you. I'm sorry to hear abou

' Defective product'

In [21]:
generate_analysis(llm=llm, transcript=transcripts[1], template=escalation_template)

This is escalation classification program. Did customer asked for escalation during the call. use following classes ["YES", "NO",  "UNKNOWN"]. Classify the conversation into one 
and exact one of these classes. Answer in one word without any explaination. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class.

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye

'NO'

In [22]:
generate_analysis(llm=llm, transcript=transcripts[1], template=holdup_template)

This is delay identification program. Did agent put customer on hold during the call. 
If Yes, provide top reason concisely. If no wait or hold, then just say "No Hold-up".

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye!"

Top Reason: 




'No Hold-up'

## Handling long call transcripts
We'll cover how to handle long transcripts that exceed the limits of the LLM. 

In [16]:
# Check if exceeds titan limit of 4000 tokens; Chunk it up
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
# Generate Analysis
def generate_analysis(llm, transcript, template="", model="Claude"):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    
    analysis_prompt = prompt.format(transcript=transcript)
    
    num_tokens = llm.get_num_tokens(analysis_prompt)
    print (f'prompt has almost {num_tokens} tokens \n')
    
    print ("max_tokens[model]", max_tokens[model])
    if num_tokens > max_tokens[model]:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n"],
            chunk_size=500,
            chunk_overlap=20
        )
        docs = text_splitter.create_documents([transcript])
        
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(
            f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
        )
        summary_chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=True) # map_reduce
        
        transcript = summary_chain.run(docs)
                
    analysis_prompt = prompt.format(transcript=transcript) 
    analysis = llm(analysis_prompt)
    
    return (analysis)

### Summary

In [19]:
generate_analysis(llm=llm, transcript=transcripts[0], template=summary_template)

prompt has almost 452 tokens 

max_tokens[model] 120
Now we have 5 documents and the first one has 101 tokens


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"timestamp: 2022-12-27 08:26:49.219717

Agent: Thank you for calling our retail support line. My name is ABC. How can I assist you today?

Customer: Yes, I have received a defective product, and I am extremely angry about it! This is unacceptable, and I want it resolved immediately!

Agent: I'm sorry to hear that you received a defective product. Can you please let me know what the issue is?"


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: 
Customer calls retail support line to report defective product and demands immediate resolution. Agent apologizes and asks for details about the 

" The customer contacts customer service to express concern about the quality of a product they purchased. The agent apologizes and offers a refund, and also mentions that the company is taking steps to ensure that this doesn't happen again in the future. The customer requests some form of compensation for the inconvenience, and the agent asks them to complete a short survey. The customer agrees to complete the survey and thanks the agent for their assistance. The agent assures the customer that they value their feedback and asks if there is anything else they can assist with. The customer declines additional assistance and thanks the agent again before ending the call."

### Sentiment Analysis

In [58]:
generate_analysis(llm=llm, transcript=transcripts[1], template=sentiment_template)


This is a sentiment analysis program. What is the customer sentiment using following classes 
["POSITIVE", "NEUTRAL","NEGATIVE"]. classify the conversation into one and exact one of these classes. 
If you don't know or not sure, please use ["NEUTRAL"] class. Do not try to make up a class.

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye!"

sentiment: 


ValueError: Error raised by bedrock service: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.

### Intent Analysis

In [ ]:
generate_analysis(llm=llm,transcript=transcripts[1], template=intent_template)

### Resolution Analysis

In [ ]:
generate_analysis(llm=llm, transcript=transcripts[1], template=resolution_template)